# TP1 - Parte 2 - Hilos

# **Java**
Este programa genera una matriz de N x N y la multiplica por un
escalar de manera secuencial (algoritmo sin hilos) y de manera concurrente (algoritmo con hilos) en Java.

In [ ]:
%%writefile Main.java
public class Main
{
    /* Cotas */
    private static final int cotaSuperior = 9;
    private static final int cotaInferior = 0;
    private static int tamanioMatriz = 80;
    /* Recursos comunes a todos */
    private static final int escalar = 1;
    /* Recursos para Hilos */
    private static final int numeroHilos=4; ////Puede ser 1, 2 o 4
    /* Recursos para comparar */
    private static int[][] rs = new int[tamanioMatriz][tamanioMatriz];
    private static int[][] rc = new int[tamanioMatriz][tamanioMatriz];
    /* Rondas de pruebas */
    private static int tamanioMuestraTesting = 1;
    private static double[] tiempoEmpleadoSecuencial = new double[tamanioMuestraTesting];
    private static double[] tiempoEmpleadoConcurrente = new double[tamanioMuestraTesting];


    public static void main(String[] args) throws Exception
    {
        inicializarMatrices();
        inicializarSecuencial();
        inicializarCalculoConcurrente();
        verificarIgualdad();
        calcularPromedioDeEjecucion();
    }

    private static void inicializarMatrices()
    {
        for(int i=0;i<tamanioMatriz;i++)
        {
            for(int j=0;j<tamanioMatriz;j++)
            {
               int numero = obtenerNumeroRandom();
               rs[i][j] = numero;
               rc[i][j] = numero;
            }
        }
    }

    private static int obtenerNumeroRandom()
    {
        int valorRandom = (int) (Math.floor(Math.random()*(cotaSuperior-cotaInferior)+cotaInferior));
        return valorRandom;
    }

    private static void inicializarCalculoConcurrente()
    {
        for(int pos = 0 ; pos < tamanioMuestraTesting ; pos++)
        {
            Concurrente cConcurrente = new Concurrente(numeroHilos, escalar, rc);
            cConcurrente.arranque();
            tiempoEmpleadoConcurrente[pos] = cConcurrente.obtenerUltimaMedicion();
        }
    }

    private static void verificarIgualdad()
    {
        boolean sonIguales = true;
        for(int i=0;i<tamanioMatriz;i++)
        {
            for(int j=0;j<tamanioMatriz;j++)
            {
                if(rs[i][j] != rc[i][j])
                {
                    sonIguales = false;
                }
            }
        }

        if(sonIguales)
        {
            System.out.println("Las matrices son iguales");
        }
        else
        {
            System.out.println("Las matrices no son iguales");
        }
    }

    private static void inicializarSecuencial()
    {
        for(int pos = 0 ; pos < tamanioMuestraTesting ; pos++)
        {
            Secuencial cSecuencial = new Secuencial(escalar, rs);
            rs=cSecuencial.calculoSecuencial();
            tiempoEmpleadoSecuencial[pos] = cSecuencial.obtenerUltimaMedicion();
        }
    }

    private static void calcularPromedioDeEjecucion()
    {
        double sumaSecuencial=0,sumaConcurrente=0;
        double promedioSecuencial,promedioConcurrente;
        for(int i=0; i<tamanioMuestraTesting; i++)
        {
            sumaSecuencial += tiempoEmpleadoSecuencial[i];
            sumaConcurrente += tiempoEmpleadoConcurrente[i];
        }
        promedioSecuencial = sumaSecuencial / tamanioMuestraTesting;
        promedioConcurrente = sumaConcurrente / tamanioMuestraTesting;
        System.out.println("El tiempo promedio de ejecucion para un calculo secuencial es de "+promedioSecuencial+" ms");
        System.out.println("El tiempo promedio de ejecucion para un calculo concurrente es de "+promedioConcurrente+" ms");
    }
}

Writing Main.java


In [ ]:
%%writefile Secuencial.java
import java.time.Duration;
import java.time.Instant;

public class Secuencial
{
    private int[][] rs;
    private int escalar;
    private long ultimaMedicion;

    public Secuencial(int valorEscalar, int[][] matrizOriginal)
    {
        escalar=valorEscalar;
        rs = matrizOriginal;
    }

    public int[][] calculoSecuencial()
    {
        Instant inicio = Instant.now();
        for(int i=0;i<rs.length;i++)
        {
            for(int j=0;j<rs.length;j++)
            {
                rs[i][j] = rs[i][j]*escalar;
            }
        }
        Instant fin = Instant.now();
        long tiempoEmpleado = Duration.between(inicio, fin).toMillis();
        ultimaMedicion = tiempoEmpleado;
        return rs;
    }

    public double obtenerUltimaMedicion()
    {
        return (double) ultimaMedicion;
    }


}

Writing Secuencial.java


In [ ]:
%%writefile Concurrente.java
import java.time.Duration;
import java.time.Instant;

public class Concurrente
{
    private int numeroHilos;
    private int escalar;
    private int filasPorHilo;
    private int filasAdicionales;
    private int[][] rc;
    private long ultimaMedicion;

    public Concurrente(int numeroHilos, int escalar,int[][] rc)
    {
        this.numeroHilos=numeroHilos;
        this.escalar=escalar;
        this.rc=rc;

        filasPorHilo = this.rc.length / this.numeroHilos;
        filasAdicionales = this.rc.length % this.numeroHilos;
    }

    public void arranque()
    {
        int filaInicial = 0,filaFinal;
        HiloMultiplicadorDeMatriz[] threads = new HiloMultiplicadorDeMatriz[numeroHilos];
        Instant inicio = Instant.now();
        for (int i = 0; i < numeroHilos; i++)
        {
            filaFinal = filaInicial + filasPorHilo + (i < filasAdicionales ? 1 : 0);
            threads[i] = new HiloMultiplicadorDeMatriz(filaInicial, filaFinal);
            threads[i].start();
            filaInicial = filaFinal;
        }

        for (int i = 0; i < numeroHilos; i++)
        {
            try
            {
                threads[i].join();
            }
            catch (InterruptedException e)
            {
                e.printStackTrace();
            }
        }
        Instant fin = Instant.now();
        long tiempoEmpleado = Duration.between(inicio, fin).toMillis();
        ultimaMedicion = tiempoEmpleado;

    }

    public double obtenerUltimaMedicion()
    {
        return (double) ultimaMedicion;
    }

    private class HiloMultiplicadorDeMatriz extends Thread
    {
        private final int startRow;
        private final int endRow;

        public HiloMultiplicadorDeMatriz(int startRow, int endRow)
        {
            this.startRow = startRow;
            this.endRow = endRow;
        }

        @Override
        public void run()
        {
            for (int i = startRow; i < endRow; i++)
            {
                for (int j = 0; j < rc.length; j++)
                {
                    rc[i][j] *= escalar;
                }
            }
        }
    }

}

Writing Concurrente.java


In [ ]:
!javac Main.java Concurrente.java Secuencial.java

In [ ]:
!java Main

Las matrices son iguales
El tiempo promedio de ejecucion para un calculo secuencial es de 0.0 ms
El tiempo promedio de ejecucion para un calculo concurrente es de 1.0 ms
